In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
sns.set(style="white",color_codes=True)
plt.rcParams['figure.figsize'] = (15,9.27)
# Set the font set of the latex code to computer modern
matplotlib.rcParams['mathtext.fontset'] = "cm"

In [2]:
mul = pd.read_excel('data/mul.xlsx')
mul.set_index(['date','id'],inplace=True)
np.random.seed(123)
mul['z'] = 2*mul.x + 3*mul.y + np.random.randn(48)

In [32]:
def df_reg(df,Y):
    X = sm.add_constant(df.drop(Y,axis=1))
    res = sm.OLS(df[Y],X).fit().params
    return res

In [4]:
def first_reg(df,Y):
    assets = df.index.levels[1]
    betas = pd.DataFrame()
    for id in assets:
        beta = df_reg(df,Y)
        betas[id] = beta
    res = betas.transpose()
    res.columns = ['const'] + ['beta_'+ x for x in df.drop(Y,axis=1).columns]
    res.index = assets
    return res

In [42]:
def second_reg(df,betas,Y):
    betas = betas.drop('const',axis=1)
    times = df.index.levels[0]
    premiums = pd.DataFrame()
    for time in times:
        yy = pd.DataFrame(df.xs((time))[Y])
        new_df = pd.merge(yy,betas,on='id',how='inner')
        print(new_df)
        print(sm.add_constant(new_df.drop(Y,axis=1)))
        premium = df_reg(new_df,Y)
        print(premium)
        premiums[time] = premium
    res = premiums.transpose()
    res.columns = ['const'] + ['premium_'+x for x in df.drop(Y,axis=1).columns]
    return res

In [6]:
[x for x in mul.drop('z',axis=1).columns]

['y', 'x']

In [7]:
betas = first_reg(mul,'z')

In [38]:
aaa = mul.xs('2012-1-1')

In [40]:
df_reg(aaa,'z')

const   -6.064287
y        3.726712
x        2.510530
dtype: float64

In [43]:
second_reg(mul,betas,'z')

            z    beta_y    beta_x
id                               
1   19.354369  3.056747  2.057594
2   26.727345  3.056747  2.057594
3   22.632978  3.056747  2.057594
4   22.713705  3.056747  2.057594
      beta_y    beta_x
id                    
1   3.056747  2.057594
2   3.056747  2.057594
3   3.056747  2.057594
4   3.056747  2.057594
beta_y    5.145934
beta_x    3.463892
dtype: float64
            z    beta_y    beta_x
id                               
1    5.431400  3.056747  2.057594
2   23.781437  3.056747  2.057594
3    8.743321  3.056747  2.057594
4   26.761087  3.056747  2.057594
      beta_y    beta_x
id                    
1   3.056747  2.057594
2   3.056747  2.057594
3   3.056747  2.057594
4   3.056747  2.057594
beta_y    3.642530
beta_x    2.451903
dtype: float64
            z    beta_y    beta_x
id                               
1   39.085936  3.056747  2.057594
2   11.573260  3.056747  2.057594
3   45.681114  3.056747  2.057594
4   22.775291  3.056747  2.057594
      

ValueError: Length mismatch: Expected axis has 2 elements, new values have 3 elements